In [1]:
import os
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torch.utils.tensorboard import SummaryWriter
import matplotlib.pyplot as plt

# Check if GPU is available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# 1. Dataset Definition

In [2]:
class FluidDataset(Dataset):
    def __init__(self, input_dir, target_dir):
        self.ux_files = sorted([f for f in os.listdir(input_dir) if f.startswith("ux")])
        self.uy_files = sorted([f for f in os.listdir(input_dir) if f.startswith("uy")])
        self.target_files = sorted(os.listdir(target_dir))
        
        # Check if file counts match
        if not (len(self.ux_files) == len(self.uy_files) == len(self.target_files)):
            raise ValueError("Input files and target files do not match in quantity.")
        
        self.input_dir = input_dir
        self.target_dir = target_dir

    def __len__(self):
        return len(self.target_files)

    def __getitem__(self, idx):
        ux = np.load(os.path.join(self.input_dir, self.ux_files[idx])).squeeze(0)
        uy = np.load(os.path.join(self.input_dir, self.uy_files[idx])).squeeze(0)
        input_data = np.concatenate((ux, uy), axis=0)

        target = np.load(os.path.join(self.target_dir, self.target_files[idx])).squeeze(0)

        return torch.tensor(input_data, dtype=torch.float32), torch.tensor(target, dtype=torch.float32)


# 2. Model Definition

In [3]:
class ResidualBlock(nn.Module):
    def __init__(self, in_channels, out_channels, stride=1, dropout_rate=0.3):
        super(ResidualBlock, self).__init__()
        self.conv1 = nn.Conv2d(in_channels, out_channels, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(out_channels)
        self.relu = nn.ReLU(inplace=True)
        self.dropout = nn.Dropout(dropout_rate)
        self.conv2 = nn.Conv2d(out_channels, out_channels, kernel_size=3, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(out_channels)

        self.shortcut = nn.Sequential()
        if stride != 1 or in_channels != out_channels:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_channels, out_channels, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(out_channels)
            )

    def forward(self, x):
        out = self.relu(self.bn1(self.conv1(x)))
        out = self.dropout(out)
        out = self.bn2(self.conv2(out))
        out += self.shortcut(x)
        return self.relu(out)

class ModelA_ResNet(nn.Module):
    def __init__(self, dropout_rate=0.3):
        super(ModelA_ResNet, self).__init__()
        self.encoder = nn.Sequential(
            nn.Conv2d(2, 64, kernel_size=7, stride=2, padding=3, bias=False),
            nn.BatchNorm2d(64),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2, padding=1),
            ResidualBlock(64, 64, dropout_rate=dropout_rate),
            ResidualBlock(64, 128, stride=2, dropout_rate=dropout_rate),
            ResidualBlock(128, 256, stride=2, dropout_rate=dropout_rate),
            ResidualBlock(256, 512, stride=2, dropout_rate=dropout_rate),
        )
        self.decoder = nn.Sequential(
            nn.ConvTranspose2d(512, 256, kernel_size=3, stride=2, padding=1, output_padding=1),
            nn.ReLU(),
            ResidualBlock(256, 256, dropout_rate=dropout_rate),
            nn.ConvTranspose2d(256, 128, kernel_size=3, stride=2, padding=1, output_padding=1),
            nn.ReLU(),
            ResidualBlock(128, 128, dropout_rate=dropout_rate),
            nn.ConvTranspose2d(128, 64, kernel_size=3, stride=2, padding=1, output_padding=1),
            nn.ReLU(),
            ResidualBlock(64, 64, dropout_rate=dropout_rate),
            nn.ConvTranspose2d(64, 4, kernel_size=3, stride=2, padding=1, output_padding=1)
        )

    def forward(self, x):
        x = self.encoder(x)
        x = self.decoder(x)
        return F.interpolate(x, size=(512, 512), mode='bilinear', align_corners=False)


# 3. Training and Testing Functions

In [ ]:
def calculate_accuracy(output, target, threshold=0.1):
    return (torch.abs(output - target) < threshold).float().mean().item()


In [ ]:
def train_model(model, dataloader, criterion, num_epochs=100, threshold=0.1, save_epochs=[1, 50, 100]):
    optimizer = optim.Adam(model.parameters(), lr=0.00001, weight_decay=1e-4)
    writer = SummaryWriter(log_dir='training_logs')
    model.train()
    
    epoch_losses, epoch_accuracies = [], []
    for epoch in range(num_epochs):
        running_loss, running_accuracy = 0.0, 0.0
        for i, (imgs, targets) in enumerate(dataloader):
            imgs, targets = imgs.to(device), targets.to(device)
            outputs = model(imgs)
            loss = criterion(outputs, targets)
            accuracy = calculate_accuracy(outputs, targets, threshold=threshold)

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            
            running_loss += loss.item()
            running_accuracy += accuracy

            if i % 10 == 0:
                writer.add_scalar('Training Loss', loss.item(), epoch * len(dataloader) + i)
                writer.add_scalar('Training Accuracy', accuracy, epoch * len(dataloader) + i)
        
        epoch_losses.append(running_loss / len(dataloader))
        epoch_accuracies.append(running_accuracy / len(dataloader))
        print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {epoch_losses[-1]:.4f}, Accuracy: {epoch_accuracies[-1]:.4f}")
        
    writer.close()
    np.save('epoch_losses.npy', np.array(epoch_losses))
    np.save('epoch_accuracies.npy', np.array(epoch_accuracies))


In [ ]:
def test_model(model, dataloader, criterion, threshold=0.1):
    model.eval()
    test_loss, test_accuracy = 0.0, 0.0
    all_predictions, all_targets = [], []

    with torch.no_grad():
        for imgs, targets in dataloader:
            imgs, targets = imgs.to(device), targets.to(device)
            outputs = model(imgs)
            loss = criterion(outputs, targets)
            accuracy = calculate_accuracy(outputs, targets, threshold=threshold)

            test_loss += loss.item()
            test_accuracy += accuracy
            all_predictions.append(outputs.cpu().numpy())
            all_targets.append(targets.cpu().numpy())

    avg_test_loss = test_loss / len(dataloader)
    avg_test_accuracy = test_accuracy / len(dataloader)
    print(f"Test Loss: {avg_test_loss:.4f}, Test Accuracy: {avg_test_accuracy:.4f}")

    # 保存预测和目标数据
    np.save('predictions.npy', np.concatenate(all_predictions, axis=0))
    np.save('targets.npy', np.concatenate(all_targets, axis=0))

    # 返回平均测试损失和准确率
    return avg_test_loss, avg_test_accuracy


# 4. Velocity Calculation Function

In [ ]:
def calculate_velocity(psi_real, psi_imag):
    da_dx, da_dy = np.gradient(psi_real[0], axis=(1, 0))
    db_dx, db_dy = np.gradient(psi_imag[0], axis=(1, 0))
    dc_dx, dc_dy = np.gradient(psi_real[1], axis=(1, 0))
    dd_dx, dd_dy = np.gradient(psi_imag[1], axis=(1, 0))

    u_x = psi_real[0] * db_dx - psi_imag[0] * da_dx + psi_real[1] * dd_dx - psi_imag[1] * dc_dx
    u_y = psi_real[0] * db_dy - psi_imag[0] * da_dy + psi_real[1] * dd_dy - psi_imag[1] * dc_dy

    return u_x, u_y

# 5. Main Training and Testing Workflow

In [ ]:
if __name__ == "__main__":
    # Directories
    train_input_dir = 'D:/zjPhD/Programzj/psitouTest/GLT_Noise_Net/data/modelA/train/input'
    train_target_dir = 'D:/zjPhD/Programzj/psitouTest/GLT_Noise_Net/data/modelA/train/target'
    test_input_dir = 'D:/zjPhD/Programzj/psitouTest/GLT_Noise_Net/data/modelA/test/input'
    test_target_dir = 'D:/zjPhD/Programzj/psitouTest/GLT_Noise_Net/data/modelA/test/target'

    # Prepare datasets and dataloaders
    train_dataset = FluidDataset(train_input_dir, train_target_dir)
    train_dataloader = DataLoader(train_dataset, batch_size=1, shuffle=True)

    test_dataset = FluidDataset(test_input_dir, test_target_dir)
    test_dataloader = DataLoader(test_dataset, batch_size=1, shuffle=False)

    # Initialize model, loss, and optimizer
    model = ModelA_ResNet().to(device)
    criterion = nn.MSELoss()

    # Train the model
    train_model(model, train_dataloader, criterion, num_epochs=500, threshold=0.1)

    # Save trained model
    torch.save(model.state_dict(), 'modelA_trained.pth')

    # Test the model
    model.load_state_dict(torch.load('modelA_trained.pth'))
    test_loss, test_accuracy = test_model(model, test_dataloader, criterion, threshold=0.1)

    # Visualization of loss and accuracy
    epoch_losses = np.load('epoch_losses.npy')
    epoch_accuracies = np.load('epoch_accuracies.npy')
    predictions = np.load('predictions.npy')
    targets = np.load('targets.npy')

    # Plotting training loss and accuracy
    fig, ax1 = plt.subplots()
    ax1.plot(epoch_losses, 'b-', label='Loss')
    ax1.set_xlabel('Epoch')
    ax1.set_ylabel('Loss', color='b')
    ax1.tick_params(axis='y', labelcolor='b')

    ax2 = ax1.twinx()
    ax2.plot(epoch_accuracies, 'r-', label='Accuracy')
    ax2.set_ylabel('Accuracy', color='r')
    ax2.tick_params(axis='y', labelcolor='r')

    plt.title('Training Loss and Accuracy')
    fig.tight_layout()
    plt.show()

    # Display sample prediction vs target
    fig, axes = plt.subplots(2, 4, figsize=(16, 8))
    titles = ['psi1_real', 'psi1_imag', 'psi2_real', 'psi2_imag']

    for i in range(4):
        # Target data
        axes[0, i].imshow(targets[0, i], cmap='RdBu_r')
        axes[0, i].set_title(f"Target {titles[i]}")
        
        # Prediction data
        axes[1, i].imshow(predictions[0, i], cmap='RdBu_r')
        axes[1, i].set_title(f"Prediction {titles[i]}")

    plt.show()